<a href="https://colab.research.google.com/github/TharinsaMudalige/Neuron-Brain_Tumor_Detection_Classification_with_XAI/blob/Detection-Classficiation-CNN/Faster_R_CNN_for_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Required Libraries

In [1]:
!pip install albumentations
!pip install cloud-tpu-client
!pip install torch_xla[tpu]  # install the necessary package for TPU support
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.13-cp310-cp310-linux_x86_64.whl

import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import xml.etree.ElementTree as ET
import albumentations as A
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

# TPU-specific libraries for PyTorch XLA
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl

# Mount Google Drive later in the file
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
     

ERROR: torch_xla-1.13-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.


/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


Define File paths

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Define dataset paths
DATASET_PATH = "/content/drive/MyDrive/DSGP/Preprocessed_Dataset"
TRAIN_PATH = os.path.join(DATASET_PATH, "Train")
TEST_PATH = os.path.join(DATASET_PATH, "Test")
VAL_PATH = os.path.join(DATASET_PATH, "Val")

# Check dataset structure
if not os.path.exists(TRAIN_PATH):
    raise FileNotFoundError(f"Train folder not found: {TRAIN_PATH}")
if not os.path.exists(TEST_PATH):
    raise FileNotFoundError(f"Test folder not found: {TEST_PATH}")

Mounted at /content/drive


Define Custom Faster R-CNN Model

In [3]:
class CustomFasterRCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomFasterRCNN, self).__init__()

        # Use a ResNet50 backbone without pretrained weights
        backbone = resnet_fpn_backbone('resnet50', pretrained=False)

        # Adjusted anchor generator
        rpn_anchor_generator = AnchorGenerator(
            sizes=([32], [64], [128], [256], [512]),
            aspect_ratios=([0.5, 1.0, 2.0],) * 5
        )

        # Define the Faster R-CNN model
        self.model = FasterRCNN(
            backbone,
            num_classes=num_classes,
            rpn_anchor_generator=rpn_anchor_generator
        )

    def forward(self, images, targets=None):
        return self.model(images, targets)

Define Dataset Class

In [4]:
class TumorDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.image_files = []
        self.annotation_files = []
        self.labels_set = set()

        images_path = os.path.join(root_dir, "Images")
        annotations_path = os.path.join(root_dir, "Annotations")

        # Ensure folders exist
        if not os.path.exists(images_path):
            raise FileNotFoundError(f"Missing 'Images' directory in {root_dir}")
        if not os.path.exists(annotations_path):
            raise FileNotFoundError(f"Missing 'Annotations' directory in {root_dir}")

        for tumor_type in sorted(os.listdir(images_path)):
            tumor_images_dir = os.path.join(images_path, tumor_type)
            tumor_annotations_dir = os.path.join(annotations_path, tumor_type)

            if not os.path.isdir(tumor_images_dir) or not os.path.isdir(tumor_annotations_dir):
                continue

            # Sort files to maintain order
            tumor_images = sorted(os.listdir(tumor_images_dir))
            tumor_annotations = sorted(os.listdir(tumor_annotations_dir))

            for image in tumor_images:
                image_path = os.path.join(tumor_images_dir, image)
                annotation_path = os.path.join(tumor_annotations_dir, os.path.splitext(image)[0] + ".xml")

                if os.path.exists(annotation_path):
                    self.image_files.append(image_path)
                    self.annotation_files.append(annotation_path)

                    # Extract labels from annotation
                    _, labels = self.parse_annotation(annotation_path)
                    self.labels_set.update(labels)

        # Create a fixed label mapping
        self.label_dict = {name: idx + 1 for idx, name in enumerate(sorted(self.labels_set))}

    def __len__(self):
        return len(self.image_files)

    def parse_annotation(self, annotation_path):
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        boxes = []
        labels = []

        for obj in root.findall("object"):
            name = obj.find("name").text
            labels.append(name)  # Tumor class name

            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)

            boxes.append([xmin, ymin, xmax, ymax])

        return np.array(boxes, dtype=np.float32), labels

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        annotation_path = self.annotation_files[idx]

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        boxes, labels = self.parse_annotation(annotation_path)
        labels = [self.label_dict[label] for label in labels]

        target = {
            "boxes": torch.tensor(boxes, dtype=torch.float32),
            "labels": labels
        }

        if self.transforms:
            # The 'class_labels' field is expected by albumentations
            sample = self.transforms(image=image, bboxes=boxes, class_labels=labels)
            image = sample["image"]
            target["boxes"] = torch.tensor(sample["bboxes"], dtype=torch.float32)

        return image, target

Define Data Transformations

In [5]:
transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.pytorch.transforms.ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

Load dataset and DataLoader

In [6]:
# Create Dataset and DataLoader
train_dataset = TumorDataset(TRAIN_PATH, transforms=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

Initialize Model and Training Setup

In [7]:
# Set device to TPU using PyTorch XLA
device = xm.xla_device()
print("Using device:", device)

# Get number of tumor classes from annotations
unique_classes = set()
for annotation in train_dataset.annotation_files:
    tree = ET.parse(annotation)
    root = tree.getroot()
    for obj in root.findall("object"):
        unique_classes.add(obj.find("name").text)

num_classes = len(unique_classes) + 1  # Tumor classes + background

# Initialize the custom Faster R-CNN model and move it to the TPU
model = CustomFasterRCNN(num_classes=num_classes)
model.to(device)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: xla:0


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train the Model

In [8]:
num_epochs = 1  # Adjust the number of epochs as needed
losses = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    # Wrap the DataLoader for TPU parallel loading
    para_loader = pl.ParallelLoader(train_loader, [device])
    for images, targets in para_loader.per_device_loader(device):
        images = [image.to(device) for image in images]
        # Convert target fields to tensors and move to TPU device
        targets = [
            {k: (torch.tensor(v).to(device) if isinstance(v, list) else v.to(device))
             for k, v in t.items()} for t in targets
        ]

        optimizer.zero_grad()
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        loss.backward()
        xm.optimizer_step(optimizer, barrier=True)

        total_loss += loss.item()

    losses.append(total_loss)
    print(f"Epoch {epoch+1}, Loss: {total_loss}")

RuntimeError: Bad StatusOr access: UNIMPLEMENTED: While rewriting computation to not contain X64 element types, XLA encountered an HLO for which this rewriting is not implemented: %rng.14 = s64[1]{0} rng(s64[] %get-tuple-element.9, s64[] %get-tuple-element.11), distribution=rng_uniform

Plot Training Loss

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), losses, marker='o', linestyle='-', color='b')
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss Over Epochs")
plt.show()

Save the Model

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/DSGP/faster_rcnn_tumor_classification.pth")
print("Model saved!")

Evaluate the Model on Test Data

In [ ]:
model.eval()
sample_image_path = "/content/drive/MyDrive/DSGP/CNN_dataset/Test/Images/astrocitoma/sample.jpg"

image = cv2.imread(sample_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Apply transform and add batch dimension
image_transformed = transform(image=image)["image"].unsqueeze(0).to(device)
output = model(image_transformed)
print("Inference output:", output)

Compute the IoU Score

In [ ]:
# Function to Calculate IoU
def calculate_iou(boxA, boxB):
    """Calculate Intersection over Union (IoU)."""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    intersection = max(0, xB - xA) * max(0, yB - yA)
    boxA_area = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxB_area = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    union = boxA_area + boxB_area - intersection

    return intersection / union if union > 0 else 0

# For demonstration, use the sample image annotation
sample_annotation_path = sample_image_path.replace("Images", "Annotations").replace(".png", ".xml")
boxes_gt, _ = train_dataset.parse_annotation(sample_annotation_path)

ious = []
# Compute IoU for each predicted box (here comparing to the first ground truth box for demonstration)
for box in output[0]['boxes']:
    iou = calculate_iou(box.cpu().detach().numpy(), boxes_gt[0])
    ious.append(iou)

# Plot IoU Scores
plt.figure(figsize=(10, 5))
plt.hist(ious, bins=20, color='g', alpha=0.7)
plt.xlabel("IoU Score")
plt.ylabel("Frequency")
plt.title("IoU Score Distribution")
plt.show()